# easy2edm

This notebook creates proceedings for the [International Conference on Educational Data Mining](https://educationaldatamining.org/conferences/) using reviewing data from [EasyChair](https://easychair.org).

See the [README](README.md) for how to download and structure the data.

In [7]:
# easy2acl.py - Convert data from EasyChair for use with ACLPUB
#
# Original Author: Nils Blomqvist
# Forked/modified by: Asad Sayeed
# Further modifications and docs (for 2019 Anthology): Matt Post
# Index for LaTeX book proceedings: Mehdi Ghanimifard and Simon Dobnik
# Modified for EDM by Andrew Olney 
# Please see the documentation in the README file at http://github.com/acl-org/easy2acl.

import os
import re
import sys

from csv import DictReader
from glob import glob
from shutil import copy, rmtree
from unicode_tex import unicode_to_tex
from pybtex.database import BibliographyData, Entry
from PyPDF2 import PdfFileReader
from functools import cmp_to_key

# Specify conference tracks  here IN PROCEEDINGS ORDER
tracks =  ['long-papers','short-papers','posters','doctoral-consortium','industry-track']

# specify decision types here
decisions = {
    "Accept in current track" : None,
    "Accept+move to short" :"short-papers",
    "Accept+move to posters":"posters",
    "ACCEPT":None
}

# for later sorting papers by track
# (submission_id, title, authors, track)
track_order = ['front'] + tracks 
def paper_cmp(a, b):
    if track_order.index(a[3]) > track_order.index(b[3]):
        return 1
    elif a[3] == b[3]:
        if int(a[0]) > int(b[0]):
            return 1
        else:
            return -1
    else:
        return -1
paper_cmp_key = cmp_to_key(paper_cmp)

def texify(string):
    """Return a modified version of the argument string where non-ASCII symbols have
    been converted into LaTeX escape codes.

    """
    return ' '.join(map(unicode_to_tex, string.split())).replace(r'\textquotesingle', "'")

def get_track_metadata(directory):
    #,----
    #| Metadata
    #`----
    metadata = { 'chairs': [] }
    with open(os.path.join(directory, 'meta')) as metadata_file:
        for line in metadata_file:
            key, value = line.rstrip().split(maxsplit=1)
            if key == 'chairs':
                metadata[key].append(value)
            else:
                metadata[key] = value

    for key in 'abbrev volume title shortbooktitle booktitle month year location publisher chairs'.split():
        if key not in metadata:
            print('Fatal: missing key "{}" from "meta" file'.format(key))
            print("Please see the documentation at https://acl-org.github.io/ACLPUB/anthology.html.")
            sys.exit(1)

    for key in "bib_url volume_name short_booktitle type".split():
        if key in metadata:
            print('Fatal: bad key "{}" in the "meta" file'.format(key))
            print("Please see the documentation at https://acl-org.github.io/ACLPUB/anthology.html.")
            sys.exit(1)

    venue = metadata["abbrev"]
    volume_name = metadata["volume"]
    year = metadata["year"]
    return metadata

def collect_track_metadata():
    metadata ={}
    for d in tracks :
        metadata[d] = get_track_metadata(d)
    return metadata

# Across all tracks, build a dictionary of submissions (which has author 
# information). We do this across tracks because some submissions have 
# decisions that move them to other tracks 

def collect_submissions_and_acceptances( decision_map, metadata ):
    submissions = {}
    for d in tracks :
        with open(os.path.join(d, 'submissions')) as submissions_file:
            for line in submissions_file:
                entry = line.rstrip().split("\t")
                submission_id = entry[0]
                authors = entry[1].replace(' and', ',').split(', ')
                title = entry[2]

                submissions[submission_id] = (title, authors)
            print("Found ", len(submissions), " submitted files in ", d)

    #
    # Append each accepted submission, as a tuple, to the 'accepted' list.
    # Order in this file is used to determine program order.
    #
    accepted = []
    for d in tracks :
        with open(os.path.join(d, 'accepted')) as accepted_file:
            for line in accepted_file:
                entry = line.rstrip().split("\t")
                # modified here to filter out the rejected files rather than doing
                # that by hand
                #if entry[-1] == 'ACCEPT':
                if entry[-1] in decision_map:
                    #print(d)
                    submission_id = entry[0]
                    title = entry[1]
                    authors = submissions[submission_id][1]
                    # if we defined an explicit mapping, use it
                    if decision_map[ entry[-1] ]:
                        track = decision_map[ entry[-1] ]
                    # otherwise we should place in current track
                    else:
                        track = d

                    accepted.append((submission_id, title, authors, track))
            print("Found ", len(accepted), " accepted files in ", d)

    # Read abstracts
    abstracts = {}
    for d in tracks :
        if os.path.exists(os.path.join(d, 'submission.csv')):
            with open(os.path.join(d, 'submission.csv')) as csv_file:
                d = DictReader(csv_file)
                for row in d:
                    abstracts[row['#']] = row['abstract']
            print('Found ', len(abstracts), 'abstracts in ',d)
        else:
            print('No abstracts available.')

    #
    # Find all relevant PDFs
    #
    venue = metadata['long-papers']["abbrev"]
    year = metadata['long-papers']["year"]
    booktitle = metadata['long-papers']['booktitle']
    chairs = metadata['long-papers']['chairs']
    
    # The PDF of the full proceedings
    full_pdf_file = 'pdf/{}_{}.pdf'.format(venue, year)
    if not os.path.exists(full_pdf_file):
        print("Fatal: could not find full volume PDF '{}'".format(full_pdf_file))
        sys.exit(1)

    # The PDF of the frontmatter
    frontmatter_pdf_file = 'pdf/{}_{}_frontmatter.pdf'.format(venue, year)
    if not os.path.exists(frontmatter_pdf_file):
        print("Fatal: could not find frontmatter PDF file '{}'".format(frontmatter_pdf_file))
        sys.exit(1)

    # File locations of all PDFs (seeded with PDF for frontmatter)
    pdfs = { '0': frontmatter_pdf_file }
    for d in tracks :
        for pdf_file in glob(os.path.join(d,'pdf/{}_{}_paper_*.pdf'.format(venue, year))):
            submission_id = pdf_file.split('_')[-1].replace('.pdf', '')
            pdfs[submission_id] = pdf_file

    # List of accepted papers (seeded with frontmatter)
    accepted.insert(0, ('0', booktitle, chairs, 'front'))
    return (submissions, accepted, abstracts, pdfs)

#
# Create Anthology tarball
#

def render_bibtex_and_track_assigned_pdf(metadata, submissions, accepted, abstracts, pdfs):
    

    # All this information is shared across tracks, so we can use long-papers
    venue = metadata['long-papers']["abbrev"]
    year = metadata['long-papers']["year"]
    booktitle = metadata['long-papers']['booktitle']
    chairs = metadata['long-papers']['chairs']
    # volume name is track name
    #volume_name = metadata['long-papers']["volume"]
    location = metadata['long-papers']["location"]
    publisher = metadata['long-papers']["publisher"]
    month= metadata['long-papers']["month"]
    
    # Create destination directories
    for dir in ['bib', 'pdf']:
        dest_dir = os.path.join('proceedings/cdrom', dir)
        if not os.path.exists(dest_dir):
            os.makedirs(dest_dir)

    # Copy over "meta" file
    print('COPYING long papers meta -> proceedings/meta', file=sys.stderr)
    copy('long-papers/meta', 'proceedings/meta')

    final_bibs = []
    start_page = 1
    acepted = accepted.sort(key=paper_cmp_key)
    for paper_id, entry in enumerate(accepted):
        #print( entry)
        submission_id, paper_title, authors, track = entry
        authors = ' and '.join(authors)
        if not submission_id in pdfs:
            print('Fatal: no PDF found for paper', paper_id, file=sys.stderr)
            sys.exit(1)

        pdf_path = pdfs[submission_id]
        dest_path = 'proceedings/cdrom/pdf/{}.{}-{}.{}.pdf'.format(year, venue,track, paper_id)

        copy(pdf_path, dest_path)
        print('COPYING', pdf_path, '->', dest_path, file=sys.stderr)

        bib_path = dest_path.replace('pdf', 'bib')
        if not os.path.exists(os.path.dirname(bib_path)):
            os.makedirs(os.path.dirname(bib_path))

        anthology_id = os.path.basename(dest_path).replace('.pdf', '')

        bib_type = 'inproceedings' if submission_id != '0' else 'proceedings'
        bib_entry = Entry(bib_type, [
            ('author', authors),
            ('title', paper_title),
            ('year', year ),
            ('month',month),
            ('address', location),
            ('publisher', publisher),
        ])

        # Add page range if not frontmatter
        if paper_id > 0:
            with open(pdf_path, 'rb') as in_:
                file = PdfFileReader(in_)
                last_page = start_page + file.getNumPages() - 1
                bib_entry.fields['pages'] = '{}--{}'.format(start_page, last_page)
                start_page = last_page + 1

        # Add the abstract if present
        if submission_id in abstracts:
            bib_entry.fields['abstract'] = abstracts.get(submission_id)

        # Add booktitle for non-proceedings entries
        if bib_type == 'inproceedings':
            bib_entry.fields['booktitle'] = booktitle

        try:
            bib_string = BibliographyData({ anthology_id: bib_entry }).to_string('bibtex')
        except TypeError as e:
            print('Fatal: Error in BibTeX-encoding paper', submission_id, file=sys.stderr)
            sys.exit(1)
        final_bibs.append(bib_string)
        with open(bib_path, 'w') as out_bib:
            print(bib_string, file=out_bib)
            print('CREATED', bib_path)

In [9]:
metadata =collect_track_metadata()

submissions, accepted, abstracts, pdfs  = collect_submissions_and_acceptances( decisions, metadata )

render_bibtex_and_track_assigned_pdf(metadata, submissions, accepted, abstracts, pdfs)

Found  90  submitted files in  long-papers
Found  146  submitted files in  short-papers
Found  166  submitted files in  posters
Found  179  submitted files in  doctoral-consortium
Found  185  submitted files in  industry-track
Found  52  accepted files in  long-papers
Found  89  accepted files in  short-papers
Found  99  accepted files in  posters
Found  108  accepted files in  doctoral-consortium
Found  112  accepted files in  industry-track
Found  90 abstracts in  <csv.DictReader object at 0x7f2dbb959ad0>
Found  146 abstracts in  <csv.DictReader object at 0x7f2dbb959a50>
Found  166 abstracts in  <csv.DictReader object at 0x7f2dbb91e050>
Found  179 abstracts in  <csv.DictReader object at 0x7f2dbb959a50>
Found  185 abstracts in  <csv.DictReader object at 0x7f2dbb91e050>


COPYING long papers meta -> proceedings/meta
COPYING pdf/EDM_2022_frontmatter.pdf -> proceedings/cdrom/pdf/2022.EDM-front.0.pdf


CREATED proceedings/cdrom/bib/2022.EDM-front.0.bib


COPYING long-papers/pdf/EDM_2022_paper_7.pdf -> proceedings/cdrom/pdf/2022.EDM-long-papers.1.pdf


CREATED proceedings/cdrom/bib/2022.EDM-long-papers.1.bib


...

CREATED proceedings/cdrom/bib/2022.EDM-industry-track.110.bib


COPYING industry-track/pdf/EDM_2022_paper_60.pdf -> proceedings/cdrom/pdf/2022.EDM-industry-track.111.pdf


CREATED proceedings/cdrom/bib/2022.EDM-industry-track.111.bib


COPYING industry-track/pdf/EDM_2022_paper_92.pdf -> proceedings/cdrom/pdf/2022.EDM-industry-track.112.pdf


CREATED proceedings/cdrom/bib/2022.EDM-industry-track.112.bib


In [37]:
accepted.sort(key=paper_cmp_key)
accepted

[('0', 'Proceedings of the 15th International Conference on Educational Data Mining', ['Tanja Mitrovic', 'Nigel Bosch'], 'front'), ('7', "Insta-Reviewer: A Data-Driven Approach for Generating Instant Feedback on Students' Project Reports", ['Qinjin Jia', 'Mitchell Young', 'Yunkai Xiao', 'Jialin Cui', 'Chengyuan Liu', 'Parvez Rashid', 'Edward Gehringer'], 'long-papers'), ('10', 'Sparse Factor Autoencoders for Item Response Theory', ['Benjamin Paaßen', 'Malwina Dywel', 'Melanie Fleckenstein', 'Niels Pinkwart'], 'long-papers'), ('12', 'Exploring Common Trends in Online Educational Experiments', ['Ethan Prihar', 'Manaal Syed', 'Korinn Ostrow', 'Stacy Shaw', 'Adam Sales', 'Neil Heffernan'], 'long-papers'), ('18', 'Designing Representations for Question Sequencing using Reinforcement Learning', ['Aqil Zainal Azhar', 'Avi Segal', 'Kobi Gal'], 'long-papers'), ('21', 'Code-DKT: A Code-based Knowledge Tracing Model for Programming Tasks', ['Yang Shi', 'Min Chi', 'Tiffany Barnes', 'Thomas Price']